## Business_Intervention_Strategy

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import os

# 1. Setup Paths
model_path = '../models/telco_churn_nn_model.keras'
data_path = '../data/telco_churn_processed.csv'
report_path = '../reports/customers_to_contact.csv'

# Ensure the reports folder exists
os.makedirs('../reports', exist_ok=True)

# 2. Load Data
try:
    df = pd.read_csv(data_path)
    print("✅ Data loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find data at {data_path}")

# 3. Re-create the split
X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Load Model
print(f"Attempting to load model from: {model_path}")
try:
    model = load_model(model_path)
    print("✅ Model loaded successfully!")
except ValueError as e:
    print("\n❌ CRITICAL ERROR: The .keras file is corrupt or not a valid model.")
    print("SOLUTION: Go back to Notebook 03, re-train the model, and run model.save() again.")
    raise e

# 5. Generate Predictions
print("Generating predictions...")
probabilities = model.predict(X_test)

# 6. Business Logic
results = X_test.copy()
results['Actual_Churn'] = y_test
results['Risk_Score'] = probabilities.flatten()

# Filter "Saveable" customers
saveable_customers = results[(results['Risk_Score'] > 0.5) & (results['Risk_Score'] < 0.8)]

# 7. Consultant Report
print(f"\n=== BUSINESS INTERVENTION REPORT ===")
print(f"Total Customers Analyzed: {len(X_test)}")
print(f"High Risk (Likely Gone):  {len(results[results['Risk_Score'] >= 0.8])}")
print(f"Safe (Loyal):             {len(results[results['Risk_Score'] <= 0.5])}")
print(f"Target for Intervention:  {len(saveable_customers)}")
print(f"Strategy: Offer 10% discount to these {len(saveable_customers)} high-value customers.")

# 8. Save Action List
saveable_customers.to_csv(report_path, index=False)
print(f"✅ Action list saved to: {report_path}")

✅ Data loaded successfully.
Attempting to load model from: ../models/telco_churn_nn_model.keras
✅ Model loaded successfully!
Generating predictions...
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

=== BUSINESS INTERVENTION REPORT ===
Total Customers Analyzed: 1405
High Risk (Likely Gone):  45
Safe (Loyal):             1127
Target for Intervention:  233
Strategy: Offer 10% discount to these 233 high-value customers.
✅ Action list saved to: ../reports/customers_to_contact.csv
